<a href="https://colab.research.google.com/github/Hugekyung/TIL/blob/master/Crawl/%EB%89%B4%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81_scrapycode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import scrapy
import re
from datetime import datetime
import pandas as pd
# from naver_news.items import NaverNewsItem
# 연합뉴스 기준
# scrapy crawl news


class CrawlYeonhabNews(scrapy.Spider):
    name = 'news'
    allowed_domains = ['news.naver.com']
    url_format = "https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=001&listType=summary&date={}&page={}"

    def __init__(self):
        start = input("시작 날짜를 입력하세요: ")
        end = input("마지막 날짜를 입력하세요: ")
        startdate =  datetime.strptime(start, "%Y-%m-%d")
        enddate =  datetime.strptime(end, "%Y-%m-%d")

        self.start_urls= []
        for cur_date in pd.date_range(startdate, enddate):
            self.start_urls.append(self.url_format.format(cur_date.strftime("%Y%m%d")))

    def start_requests(self):
        for start_url in self.start_urls:
            # timer = round((time.time() - self.starttime) / 60)
            # percentage = round(self.request_count / len(self.inner_id_ls) * 100, 2)
            # print("{}분 경과, {}개 중 {}개 request 완료({}%)".format(timer, self.request_count, len(self.inner_id_ls), percentage))
            yield scrapy.Request(url=start_url, callback=self.parse)

    def parse(self, response):
        for item in response.css("ul.type06_headline"):
            # upload_date = item.css("span.date").get()
            url = item.css("a::attr(href)").get()
            yield scrapy.Request(url, callback=self.parse_detail)

        for item2 in response.css("ul.type06"):
            url = item2.css("a::attr(href)").get()
            yield scrapy.Request(url2, callback=self.parse_detail)

        # 마지막 페이지 개수 어떻게 세지?
        next_page = response.css('a.paging::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)
    
    def parse_detail(self, response):   
        upload_date = response.css("div.sponsor span.t11::text").get()
        title = response.css("h3#articleTitle::text").get()
        # media = response.css("div.press_logo img::attr(title)").get()
        text = ''.join(response.css("div#articleBodyContents::text").getall()).replace("\n","").strip()
        # summary_bot = 요약봇 결과가 서버에 다녀오는 형식이라 클릭해야 데이터가 온다 어쩌지? css말고 json형식으로 가져오는건가?
        # category = 

        yield {
                    "upload_date": upload_date,
                    "title": title,
                    "text": text,
                    "summary_bot": summary_bot,
                    "category": category,
                }